<a href="https://colab.research.google.com/github/pyt3r/springboard-package/blob/master/assignments/Student_MLE_MiniProject_Churn_Prediction/standalone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Mount Colab Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import pathlib
root = pathlib.Path('/content/drive/MyDrive/home/Research/Springboard/Colab Notebooks')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Imports

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix

## Read Data

In [3]:
df = pd.read_csv( root / 'storedata_total.csv' )
df.head()

,custid,retained,created,firstorder,lastorder,esent,eopenrate,eclickrate,avgorder,ordfreq,paperless,refill,doorstep,favday,city
0,6H6T6N,0,2012-09-28,2013-08-11 00:00:00,2013-08-11 00:00:00,29,100.000000,3.448276,14.52,0.000000,0,0,0,Monday,DEL
1,APCENR,1,2010-12-19,2011-04-01 00:00:00,2014-01-19 00:00:00,95,92.631579,10.526316,83.69,0.181641,1,1,1,Friday,DEL
2,7UP6MS,0,2010-10-03,2010-12-01 00:00:00,2011-07-06 00:00:00,0,0.000000,0.000000,33.58,0.059908,0,0,0,Wednesday,DEL
3,7ZEW8G,0,2010-10-22,2011-03-28 00:00:00,2011-03-28 00:00:00,0,0.000000,0.000000,54.96,0.000000,0,0,0,Thursday,BOM
4,8V726M,1,2010-11-27,2010-11-29 00:00:00,2013-01-28 00:00:00,30,90.000000,13.333333,111.91,0.008850,0,0,0,Monday,BOM


## Feature Engineering Transformer

We'll create a custom transformer that encapsulates all feature engineering logic. This ensures consistency between training and inference.


In [4]:

class FeatureEngineeringTransformer(BaseEstimator, TransformerMixin):
    """ Custom transformer that performs feature engineering on raw customer data. """

    def __init__(self):
        self.reference_date_ = None

    def fit(self, X, y=None):
        """ Learn the reference date from training data (max of lastorder column). """
        X = X.copy()
        X = self.to_datetime(X)

        # Store the reference date (max of lastorder) for use in transform
        self.reference_date_ = X['lastorder'].max()

        return self

    def transform(self, X):
        """ Apply feature engineering transformations to new data. """
        X = X.copy()
        X = self.to_datetime(X)

        # daysSinceLastOrder --> customers who haven't ordered recently are at high risk
        X['daysSinceLastOrder'] = (self.reference_date_ - X['lastorder']).dt.days

        # daysToFirstOrder --> customers who order quickly after signup are more engaged
        X['daysToFirstOrder'] = (X['firstorder'] - X['created']).dt.days

        # ordersPerDay
        customerLifetimeDays = (X['lastorder'] - X['created']).dt.days
        X['ordersPerDay'] = X['ordfreq'] / (customerLifetimeDays + 1)  # +1 to avoid division by zero

        # emailEngagementScore --> weighted combo of open and click rates
        X['emailEngagementScore'] = (X['eopenrate'] * 0.6 + X['eclickrate'] * 0.4)

        # serviceAdoptionScore --> customers using multiple services are more engaged
        X['serviceAdoptionScore'] = X['paperless'] + X['refill'] + X['doorstep']

        # Select only the feature columns
        feature_cols = [
            'daysSinceLastOrder',
            'daysToFirstOrder',
            'ordersPerDay',
            'emailEngagementScore',
            'serviceAdoptionScore',
            'avgorder',
            'ordfreq',
            'eopenrate',
            'eclickrate',
            'esent',
            'paperless',
            'refill',
            'doorstep',
        ]

        return X[feature_cols]

    @staticmethod
    def to_datetime(X):
        X['created'] = pd.to_datetime(X['created'], errors='coerce')
        X['firstorder'] = pd.to_datetime(X['firstorder'], errors='coerce')
        X['lastorder'] = pd.to_datetime(X['lastorder'], errors='coerce')
        return X


## Preprocess

In [5]:
copy = df.copy()
copy = FeatureEngineeringTransformer.to_datetime( copy )
copy = copy.dropna()

print( 'shape before drop:', df.shape )
df = df.loc[ copy.index ]
print( 'shape after drop:', df.shape )

shape before drop: (30801, 15)
shape after drop: (30747, 15)


## Data Splitting

First, we'll split the data into train, validation, and test sets. This ensures we have consistent splits before applying feature engineering.

In [6]:
# Split raw data into train, test, and validation sets before feature engineering
# The validation set will be used in batch inference later on.

# Prepare raw data
X_raw = df.drop('retained', axis=1)
y_raw = df['retained']

# First split: 80% train+val, 20% test
X_temp_raw, X_test_raw, y_temp_raw, y_test_raw = train_test_split(
    X_raw, y_raw,
    test_size=0.2,
    random_state=42,
    stratify=y_raw
)

# Second split: 80% train, 20% validation (from the 80% we kept)
X_train_raw, X_val_raw, y_train_raw, y_val_raw = train_test_split(
    X_temp_raw, y_temp_raw,
    test_size=0.2,  # 20% of 80% = 16% of total data
    random_state=42,
    stratify=y_temp_raw
)

print(f"Raw data split:")
print(f"Training set: {X_train_raw.shape[0]} samples ({X_train_raw.shape[0]/len(df)*100:.1f}%)")
print(f"Validation set: {X_val_raw.shape[0]} samples ({X_val_raw.shape[0]/len(df)*100:.1f}%)")
print(f"Test set: {X_test_raw.shape[0]} samples ({X_test_raw.shape[0]/len(df)*100:.1f}%)")
print(f"\nTraining set churn rate: {(1 - y_train_raw.mean()):.2%}")
print(f"Validation set churn rate: {(1 - y_val_raw.mean()):.2%}")
print(f"Test set churn rate: {(1 - y_test_raw.mean()):.2%}")

Raw data split:
Training set: 19677 samples (64.0%)
Validation set: 4920 samples (16.0%)
Test set: 6150 samples (20.0%)

Training set churn rate: 20.54%
Validation set churn rate: 20.53%
Test set churn rate: 20.54%


In [7]:
featureCols = [

    # engineered:
    'daysSinceLastOrder',
    'daysToFirstOrder',
    'ordersPerDay',
    'emailEngagementScore',
    'serviceAdoptionScore',

    # original:
    'avgorder',
    'ordfreq',
    'eopenrate',
    'eclickrate',
    'esent',
    'paperless',
    'refill',
    'doorstep',
]

## Train

We'll use the combined train+test data for model training and hyperparameter tuning. All pipelines include feature engineering to ensure consistency and production readiness. The validation set is reserved for inference.

In [8]:
rf_pipeline = Pipeline(steps=[
    ("feature_engineering", FeatureEngineeringTransformer()),
    ("scaler", StandardScaler()),
    ("classifier", RandomForestClassifier(
        n_estimators      = 100,
        max_depth         = 10,
        min_samples_split = 5,
        min_samples_leaf  = 2,
        random_state      = 42,
        n_jobs            = -1
    ))
])


rf_pipeline.fit(X_train_raw, y_train_raw)


Pipeline(steps=[('feature_engineering', FeatureEngineeringTransformer()),
                ('scaler', StandardScaler()),
                ('classifier',
                 RandomForestClassifier(max_depth=10, min_samples_leaf=2,
                                        min_samples_split=5, n_jobs=-1,
                                        random_state=42))])

In [9]:
print("Feature Importance (Random Forest):")

rf_model = rf_pipeline.named_steps['classifier']
feature_importance = pd.DataFrame({
    'feature'    : featureCols,
    'importance' : rf_model.feature_importances_
}).sort_values('importance', ascending=False)

feature_importance


Feature Importance (Random Forest):


,feature,importance
9,esent,0.679110
7,eopenrate,0.091198
3,emailEngagementScore,0.080398
0,daysSinceLastOrder,0.050262
8,eclickrate,0.035545
5,avgorder,0.012793
1,daysToFirstOrder,0.012569
4,serviceAdoptionScore,0.010787
2,ordersPerDay,0.009304
6,ordfreq,0.008523


## Tuning

In [10]:
param_grid = {
    'classifier__n_estimators'      : [50, 100],
    'classifier__max_depth'         : [5, 10, 15],
    'classifier__min_samples_split' : [5, 10],
    'classifier__min_samples_leaf'  : [2, 4]
}

rf_base_pipeline = Pipeline(steps=[
    ("feature_engineering", FeatureEngineeringTransformer()),
    ("scaler", StandardScaler()),
    ("classifier", RandomForestClassifier(
        random_state = 42,
        n_jobs       = -1
    ))
])

print("Performing grid search...")
grid_rf = GridSearchCV(
    estimator          = rf_base_pipeline,
    param_grid         = param_grid,
    cv                 = 5,
    scoring            = 'roc_auc',
    n_jobs             = -1,
    refit              = True,
    return_train_score = True,
    verbose            = 1
)

grid_rf.fit(X_train_raw, y_train_raw)
print("\ndone.")
print(f"Best parameters: {grid_rf.best_params_}")
print(f"Best cross-validation AUC ROC: {grid_rf.best_score_:.4f}")


Performing grid search...
Fitting 5 folds for each of 24 candidates, totalling 120 fits

done.
Best parameters: {'classifier__max_depth': 15, 'classifier__min_samples_leaf': 2, 'classifier__min_samples_split': 5, 'classifier__n_estimators': 100}
Best cross-validation AUC ROC: 0.9811


In [11]:
best_rf_pipeline = grid_rf.best_estimator_

print("Best Model from Grid Search:")
print(f"Best parameters: {grid_rf.best_params_}")
print(f"Best cross-validation AUC ROC: {grid_rf.best_score_:.4f}")

Best Model from Grid Search:
Best parameters: {'classifier__max_depth': 15, 'classifier__min_samples_leaf': 2, 'classifier__min_samples_split': 5, 'classifier__n_estimators': 100}
Best cross-validation AUC ROC: 0.9811


In [12]:
print("Grid Search Results Summary:")
print(f"Best parameters found: {grid_rf.best_params_}")
print(f"Best cross-validation AUC ROC: {grid_rf.best_score_:.4f}")
print(f"\nNumber of parameter combinations tested: {len(grid_rf.cv_results_['params'])}")

results_df = pd.DataFrame(grid_rf.cv_results_)
top_results = results_df.nlargest(5, 'mean_test_score')[
    ['param_classifier__n_estimators', 'param_classifier__max_depth',
     'param_classifier__min_samples_split', 'param_classifier__min_samples_leaf',
     'mean_test_score', 'std_test_score']
]
print("\nTop 5 Parameter Combinations:")
top_results


Grid Search Results Summary:
Best parameters found: {'classifier__max_depth': 15, 'classifier__min_samples_leaf': 2, 'classifier__min_samples_split': 5, 'classifier__n_estimators': 100}
Best cross-validation AUC ROC: 0.9811

Number of parameter combinations tested: 24

Top 5 Parameter Combinations:


,param_classifier__n_estimators,param_classifier__max_depth,param_classifier__min_samples_split,param_classifier__min_samples_leaf,mean_test_score,std_test_score
17,100,15,5,2,0.981092,0.002512
23,100,15,10,4,0.980634,0.003771
19,100,15,10,2,0.980469,0.003164
21,100,15,5,4,0.980412,0.003053
22,50,15,10,4,0.980336,0.003407


## Batch Inference Pipeline

All pipelines now include feature engineering and can handle raw data directly without manual preprocessing.

In [13]:
def batch_inference(pipeline, new_samples_df):
    """ Perform batch inference on new samples using the full pipeline. """

    results_df = new_samples_df.copy()
    predictions = pipeline.predict(new_samples_df)
    probabilities = pipeline.predict_proba(new_samples_df)[:, 1]

    results_df['predicted_retained'] = predictions
    results_df['churn_probability'] = 1 - probabilities
    results_df['retention_probability'] = probabilities

    return results_df

In [14]:
print("Validation Set for Batch Inference:")
print(f"Shape: {X_val_raw.shape}")
print(f"\nFirst few samples:")
print(X_val_raw.head())

Validation Set for Batch Inference:
Shape: (4920, 14)

First few samples:
       custid     created           firstorder            lastorder  esent  \
10844  F3MZPB  2012-09-20  2012-09-20 00:00:00  2012-09-20 00:00:00     36   
21185  B2NYTW  2013-11-21  2013-11-21 00:00:00  2013-11-21 00:00:00     29   
21480  BRTTYS  2013-11-25  2013-11-25 00:00:00  2013-11-25 00:00:00     29   
26371  LXDKB4  2016-06-06  2016-06-06 00:00:00  2016-06-06 00:00:00     45   
6497   VEZKPA  2011-06-04  2011-06-16 00:00:00  2012-04-02 00:00:00     47   

       eopenrate  eclickrate  avgorder   ordfreq  paperless  refill  doorstep  \
10844  50.000000    0.000000     62.50  0.000000          0       0         0   
21185   3.448276    0.000000     39.69  0.000000          1       0         0   
21480   6.896552    3.448276      1.00  0.000000          1       0         0   
26371   4.444444    2.222222     36.24  0.000000          1       0         0   
6497    6.382979    4.255319     55.50  0.041237    

In [15]:
inference_results = batch_inference(best_rf_pipeline, X_val_raw)

print(f"\nBatch Inference Results (first 10 samples):\n")
cols = ['custid', 'predicted_retained', 'churn_probability', 'retention_probability']
inference_results[cols].head(10)


Batch Inference Results (first 10 samples):



,custid,predicted_retained,churn_probability,retention_probability
10844,F3MZPB,1,0.013206,0.986794
21185,B2NYTW,1,0.005111,0.994889
21480,BRTTYS,1,0.013054,0.986946
26371,LXDKB4,1,0.006234,0.993766
6497,VEZKPA,1,0.007361,0.992639
10440,NFG42K,1,0.086682,0.913318
17375,2NFXNA,1,0.077419,0.922581
28275,RMXZWD,1,0.005424,0.994576
25179,6UXMSZ,1,0.037127,0.962873
16380,CURHHY,1,0.000402,0.999598


In [16]:
print("\n\nBatch Inference Summary:\n")

print(f"Total samples: {len(inference_results)}")
print(f"Predicted to churn: {(inference_results['predicted_retained'] == 0).sum()}")
print(f"Predicted to be retained: {(inference_results['predicted_retained'] == 1).sum()}")
print(f"\nAverage churn probability: {inference_results['churn_probability'].mean():.4f}")
print(f"Average retention probability: {inference_results['retention_probability'].mean():.4f}")

print(f"\nActual churn rate in validation set: {(1 - y_val_raw.mean()):.2%}")
print(f"Predicted churn rate: {(1 - inference_results['predicted_retained'].mean()):.2%}")

val_accuracy = (inference_results['predicted_retained'] == y_val_raw.values).mean()
print(f"\nAccuracy on validation set: {val_accuracy:.4f}")



Batch Inference Summary:

Total samples: 4920
Predicted to churn: 924
Predicted to be retained: 3996

Average churn probability: 0.2070
Average retention probability: 0.7930

Actual churn rate in validation set: 20.53%
Predicted churn rate: 18.78%

Accuracy on validation set: 0.9654


In [17]:
print("\nDetailed Results with Original Features (first 10 samples):")
display_cols = [ 'custid', 'churn_probability', 'retention_probability', 'predicted_retained']

inference_results_display = inference_results[display_cols].copy()
inference_results_display['actual_retained'] = y_val_raw.values

inference_results_display.head(10)


Detailed Results with Original Features (first 10 samples):


,custid,churn_probability,retention_probability,predicted_retained,actual_retained
10844,F3MZPB,0.013206,0.986794,1,1
21185,B2NYTW,0.005111,0.994889,1,1
21480,BRTTYS,0.013054,0.986946,1,1
26371,LXDKB4,0.006234,0.993766,1,1
6497,VEZKPA,0.007361,0.992639,1,1
10440,NFG42K,0.086682,0.913318,1,1
17375,2NFXNA,0.077419,0.922581,1,1
28275,RMXZWD,0.005424,0.994576,1,1
25179,6UXMSZ,0.037127,0.962873,1,1
16380,CURHHY,0.000402,0.999598,1,1
